In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from deep_convnet import DeepConvNet
%matplotlib inline

from common.functions import softmax

import time

Using TensorFlow backend.


In [2]:
def generate_stack_list(x_list, t_list, max_n):
    stack_x_list = []
    stack_i_list = []
    for _ in range(10):
        stack_x_list.append([])
        stack_i_list.append([])
        
    fill_list = []
    for i, t in enumerate(t_list):
        if (not t in fill_list) and len(stack_x_list[t]) < max_n:
            stack_x_list[t].append(x_list[i].flatten())
            stack_i_list[t].append(i)
        else:
            fill_list.append(t)
            
    return np.array(stack_x_list), np.array(stack_i_list)

In [3]:
def generate_avg_list(stack_x_list):            
    avg_list = []
    for stacks in stack_x_list:
        avg = np.mean(stacks, axis=0)
        avg_list.append(avg.reshape(1,28,28))
        
    return np.array(avg_list)

In [4]:
def show_pred(x, network):
    pre = network.predict(x.reshape(1,1,28,28))
    pre_label = np.argmax(pre)
    pre_score = round(max(softmax(pre[0])), 2)

    plt.imshow(x.reshape(28,28), 'gray')
    plt.title(f"{pre_label} : {pre_score}", fontsize=20)
    plt.show()

In [5]:
def generate_setsudo(x, label, network, eps):
    d, g = network.gradient_for_fgsm(x.reshape(1,1,28,28), np.array([label]))
    p = eps * np.sign(d)
    
    return p.reshape(28,28)

In [6]:
def attack_fgsm(x, label, network, eps):
    x = x.reshape(28,28)
    p = generate_setsudo(x, label, network, eps)
    adv = (x + p).clip(min=0, max=1)
    
    return adv.reshape(1,28,28)

In [7]:
def attack_fgsm_for_stack_list(stack_x_list, network, eps):
    stack_adv_list = []
    for label, stacks in enumerate(stack_x_list):
        adv_list = []
        for x in stacks:            
#             show_pred(x, network)
            adv = attack_fgsm(x, label, network, eps)
#             show_pred(adv, network)
            adv_list.append(adv)
            
        stack_adv_list.append(adv_list)
        
    return np.array(stack_adv_list)

In [8]:
def attack_random_for_stack_list(stack_x_list, network, eps):
    random_noise = np.random.normal(
        loc   = 0,      # 平均
        scale = 1,      # 標準偏差
        size  = (28, 28),# 出力配列のサイズ
    )
    
#     plt.hist(random_noise,bins=5)
#     plt.xlim(-5,5)
    
    stack_adv_list = []
    for label, stacks in enumerate(stack_x_list):
        advs = (stack_x_list[label].reshape(-1, 1, 28, 28) + np.sign(random_noise) * eps).clip(min=0, max=1)
        stack_adv_list.append(advs)
            
    return np.array(stack_adv_list)

In [9]:
def acc_stack_list(stack_x_list, network):
    for label, stacks in enumerate(stack_x_list):
        acc = 0
        for x in stacks:
#             show_pred(stacks[0], network)
            pre = network.predict(x.reshape(1,1,28,28))
            pre_label = np.argmax(pre)
            if pre_label == label:
                acc += 1
        print(label, acc/len(stacks))
        print("-----")

In [10]:
def attack_avg_for_stack_list(stack_x_list, network, eps):
    stack_adv_list = []
    stack_avg_list = generate_avg_list(stack_x_list)
    for label, avg in enumerate(stack_avg_list):
        p = generate_setsudo(avg, label, network, eps)
        advs = (stack_x_list[label].reshape(-1, 1, 28, 28) + p).clip(min=0, max=1)
        stack_adv_list.append(advs)
        
    return np.array(stack_adv_list)

In [11]:
network = DeepConvNet()

network.load_params("deep_convnet_params.pkl")
print("loaded Network Parameters!")

loaded Network Parameters!


In [12]:
eps = 0.3

In [13]:
n = 2

In [14]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = mnist.load_data()

In [15]:
stack_x_list, stack_i_list = generate_stack_list(x_test, t_test, n) 

In [16]:
stack_x_list = stack_x_list.astype(np.float32) / 255.0
stack_x_list.shape

(10, 2, 784)

In [17]:
start = time.time()
stack_adv_list = attack_fgsm_for_stack_list(stack_x_list, network, eps)
elapsed_time = time.time() - start
print (f"fgsm : {elapsed_time}[sec]")

fgsm : 0.2171182632446289[sec]


In [18]:
start = time.time()
stack_avg_adv_list = attack_avg_for_stack_list(stack_x_list, network, eps)
elapsed_time = time.time() - start
print (f"avg_fgsm : {elapsed_time}[sec]")

avg_fgsm : 0.12549400329589844[sec]


In [19]:
start = time.time()
stack_random_list = attack_random_for_stack_list(stack_x_list, network, eps)
elapsed_time = time.time() - start
print (f"random : {elapsed_time}[sec]")

random : 0.0012788772583007812[sec]


In [20]:
print("normal")
acc_stack_list(stack_x_list.reshape(10, -1, 1, 28, 28), network)

normal
0 1.0
-----
1 1.0
-----
2 1.0
-----
3 1.0
-----
4 1.0
-----
5 1.0
-----
6 1.0
-----
7 1.0
-----
8 1.0
-----
9 1.0
-----


In [21]:
print("random")
acc_stack_list(stack_random_list, network)

random
0 1.0
-----
1 1.0
-----
2 1.0
-----
3 1.0
-----
4 0.5
-----
5 1.0
-----
6 1.0
-----
7 1.0
-----
8 1.0
-----
9 1.0
-----


In [22]:
print("FGSM")
acc_stack_list(stack_adv_list, network)

FGSM
0 0.0
-----
1 0.0
-----
2 1.0
-----
3 0.5
-----
4 0.0
-----
5 0.0
-----
6 0.5
-----
7 0.0
-----
8 0.0
-----
9 0.0
-----


In [23]:
print("avg_fgsm")
acc_stack_list(stack_avg_adv_list, network)

avg_fgsm
0 0.5
-----
1 0.0
-----
2 1.0
-----
3 0.5
-----
4 0.0
-----
5 0.5
-----
6 0.5
-----
7 1.0
-----
8 0.0
-----
9 0.0
-----


In [24]:
import keras
from keras.models import model_from_json
from keras.utils import np_utils
from keras.optimizers import RMSprop

In [25]:
# モデルを読み込む
model = model_from_json(open('mnist_mlp_model.json').read())

# 学習結果を読み込む
model.load_weights('mnist_mlp_weights.h5')

model.summary();

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

W1224 14:58:27.835322 140737019925440 deprecation_wrapper.py:119] From /Users/ryuto/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1224 14:58:27.857506 140737019925440 deprecation_wrapper.py:119] From /Users/ryuto/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1224 14:58:27.878449 140737019925440 deprecation_wrapper.py:119] From /Users/ryuto/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1224 14:58:27.879408 140737019925440 deprecation_wrapper.py:119] From /Users/ryuto/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placehol

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [26]:
def acc_keras_stack_list(stack_x_list, model):
    for label, stacks in enumerate(stack_x_list):
        labels = keras.utils.to_categorical(np.full(len(stacks), label), 10)
        score = model.evaluate(stacks.reshape(-1, 784), labels, verbose=0)
        print(label, round(score[1], 2))
        print("-----")

In [27]:
acc_keras_stack_list(stack_adv_list, model)

0 1.0
-----
1 0.0
-----
2 1.0
-----
3 0.5
-----
4 0.0
-----
5 0.5
-----
6 1.0
-----
7 0.0
-----
8 1.0
-----
9 0.0
-----


In [28]:
acc_keras_stack_list(stack_x_list, model)

0 1.0
-----
1 1.0
-----
2 1.0
-----
3 1.0
-----
4 1.0
-----
5 1.0
-----
6 1.0
-----
7 1.0
-----
8 1.0
-----
9 1.0
-----


In [29]:
acc_keras_stack_list(stack_random_list, model)

0 1.0
-----
1 0.0
-----
2 1.0
-----
3 0.5
-----
4 0.0
-----
5 1.0
-----
6 1.0
-----
7 1.0
-----
8 1.0
-----
9 0.5
-----


In [30]:
acc_keras_stack_list(stack_avg_adv_list, model)

0 1.0
-----
1 0.0
-----
2 1.0
-----
3 0.5
-----
4 0.0
-----
5 0.5
-----
6 1.0
-----
7 0.0
-----
8 0.5
-----
9 0.0
-----
